pip install torchtext

In [1]:
import torch
from torchtext.vocab import GloVe

ModuleNotFoundError: No module named 'torchtext'

In [40]:
class cos_sim:
    def __init__(self):
        self.glove = GloVe(name='6B', dim=100)

    # def get_word_embedding(self, word):
    #     return self.glove(word)
    
    def get_sentence_embedding(self, phrase: str):
        words = phrase.split()
        embeddings = [self.glove[word] for word in words if word in self.glove.stoi]
        if embeddings:
            return torch.mean(torch.stack(embeddings), dim=0)
        else:
            return torch.zeros(self.glove.dim)

    def generate_cosine_similarity(
        self,
        predictions: list[str], 
        truth: str, 
        ):
        similarities = []
        for prediction in predictions:
            prediction_embeddings = self.get_sentence_embedding(prediction)
            truth_embeddings = self.get_sentence_embedding(truth)

            similarity = torch.nn.functional.cosine_similarity(prediction_embeddings, truth_embeddings, dim=0)
            similarities.append(similarity)
    
        closest_match = max(similarities)
        index_of_closest_match = similarities.index(max(similarities))

        return similarities, {'similarity': closest_match, 'index': index_of_closest_match, 'caption': predictions[index_of_closest_match]}

In [41]:
cos_sim_generator = cos_sim()

In [48]:
truth = "red helicopter"
predictions = ["red jet", "gray helicopter", "red helicopter"]

In [49]:
cos_sim_generator.generate_cosine_similarity(predictions=predictions, truth=truth)

([tensor(0.8939), tensor(0.8951), tensor(0.7161)],
 {'similarity': tensor(0.8951), 'index': 1, 'caption': 'gray helicopter'})